<a href="https://colab.research.google.com/github/ShaifaliKhulbe/Masters-Thesis-Authorship-Attributon/blob/main/French_Baseline%2BPOS(POET_Tagger).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://huggingface.co/qanastek/pos-french?text=George+Washington+est+all%C3%A9+%C3%A0+Washington



In [ ]:
!pip install flair

In [ ]:
import re
import pandas as pd
import numpy as np
import nltk
import collections
from nltk import word_tokenize
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# Load the model
model = SequenceTagger.load("qanastek/pos-french")

2023-05-12 00:22:45,295 SequenceTagger predicts: Dictionary with 69 tags: <unk>, O, DET, NFP, ADJFP, AUX, VPPMS, ADV, PREP, PDEMMS, NMS, COSUB, PINDMS, PPOBJMS, VERB, DETFS, NFS, YPFOR, VPPFS, PUNCT, DETMS, PROPN, ADJMS, PPER3FS, ADJFS, COCO, NMP, PREL, PPER1S, ADJMP, VPPMP, DINTMS, PPER3MS, PPER3MP, PREF, ADJ, DINTFS, CHIF, XFAMIL, PRELFS, SYM, NOUN, MOTINC, PINDFS, PPOBJMP, NUM, PREFP, PDEMFS, VPPFP, PPER3FP


In [ ]:
import nltk
from itertools import chain
from nltk.util import ngrams
from collections import Counter

# load data
df = pd.read_csv('200_chunks_French.csv')
test_df = pd.read_csv('Test_200_chunks_French.csv')

#define the dictionary to store all top ngrams for each author
top_ngrams = {}

#define the number of top pos ngrams to calculate
num = 60

# calculate the pos ngrams for each author and store them in pos_ngrams_by_author
for author in df['Author_name'].unique():
    author_data = df[df['Author_name'] == author]
    sentences = list(author_data['chunks'])

    all_pos_1grams = []
    all_pos_2grams = []
    all_pos_3grams = []

    for sentence in sentences:

        # tokenize the sentence
        #tokens = nltk.word_tokenize(sentence, language='french')

        # tag the tokens with POS tags

        sentence = Sentence(sentence)
        # Predict tags
        model.predict(sentence)

        tagged_string = str(sentence.to_tagged_string())

        #tagged_string = 'Sentence[6]: "George Washington est allé à Washington" → ["George"/PROPN, "Washington"/XFAMIL, "est"/AUX, "allé"/VPPMS, "à"/PREP, "Washington"/PROPN]'
        tagged_list = []

        # Extract word and POS tag pairs using regular expressions
        pattern = r'"([^"]+)"/(\w+)'
        matches = re.findall(pattern, tagged_string)

        # Iterate over matches and append word and POS tag to tagged_list
        for match in matches:
            word, pos = match
            tagged_list.append((word, pos))

        #print(tagged_list)

        pos_tags = tagged_list

        #pos_tags = nltk.pos_tag(tokens, tagset = 'universal')

        # create POS n-grams of size n
        pos_unigram_list = ngrams([tag for _, tag in pos_tags], 1)
        pos_bigram_list = ngrams([tag for _, tag in pos_tags], 2)
        pos_trigram_list = ngrams([tag for _, tag in pos_tags], 3)

        # add the POS n-grams to the list
        all_pos_1grams.extend(pos_unigram_list)
        all_pos_2grams.extend(pos_bigram_list)
        all_pos_3grams.extend(pos_trigram_list)
      
  
    # count the frequency of each POS n-gram
    pos_1gram_counts = nltk.FreqDist(all_pos_1grams)
    pos_2gram_counts = nltk.FreqDist(all_pos_2grams)
    pos_3gram_counts = nltk.FreqDist(all_pos_3grams)
    
    # return the top num POS n-grams

    top_unigrams= [ngram for ngram, count in pos_1gram_counts.most_common(num)]

    top_bigrams= [ngram for ngram, count in pos_2gram_counts.most_common(num)]

    top_trigrams= [ngram for ngram, count in pos_3gram_counts.most_common(num)]

    top_ngrams[author] = top_trigrams + top_bigrams + top_unigrams

#print(top_ngrams)

# Find the name of the first key in the dictionary
first_key = list(top_ngrams.keys())[0]

# Initialize the common_values set using the first key
common_ngrams = set(top_ngrams[first_key])

for key in top_ngrams:
    common_ngrams = common_ngrams.intersection(top_ngrams[key])

print(len(common_ngrams))

# Create a list of all values from all keys minus the common values
all_ngrams = []

for key in top_ngrams:
    all_ngrams.extend(top_ngrams[key])
    
print(len(all_ngrams))

not_common_ngrams = set(all_ngrams).difference(common_ngrams)
not_common_ngrams_list = list(not_common_ngrams)


#Finding top 100 most frequent words
def find_top_function_words(data, num):
    
    # tokenize each sentence and create a list of all words
    words = []
    for sentence in data['chunks']:
        tokens = nltk.word_tokenize(sentence.lower(), language='french')
        words.extend(tokens)

    # count the frequency of each word in the entire CSV
    word_freq = Counter(words)
    
    # sort the dictionary by frequency in descending order
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

    # extract the 100 most frequent stopwords
    
    top_stop_words = [word[0] for word in sorted_word_freq if word[0] in nltk.corpus.stopwords.words('french')][:100]

    return top_stop_words
       
#NEW    
top_function_words = find_top_function_words(df, 100)

# define a function to create the feature vector for a given sentence
def create_feature_vector(sentence):
    
    # tag the sentence with POS tags

    sent = Sentence(sentence)
    # Predict tags
    model.predict(sent)

    tagged_string = str(sent.to_tagged_string())

    #tagged_string = 'Sentence[6]: "George Washington est allé à Washington" → ["George"/PROPN, "Washington"/XFAMIL, "est"/AUX, "allé"/VPPMS, "à"/PREP, "Washington"/PROPN]'
    tagged_list = []

    # Extract word and POS tag pairs using regular expressions
    pattern = r'"([^"]+)"/(\w+)'
    matches = re.findall(pattern, tagged_string)

    # Iterate over matches and append word and POS tag to tagged_list
    for match in matches:
        word, pos = match
        tagged_list.append((word, pos))

    #print(tagged_list)

    pos_tags = tagged_list
    
    # create bigram, trigram, and four-gram POS sequences
    unigram_pos_seqs = ngrams([tag for _, tag in pos_tags], 1)
    bigram_pos_seqs = ngrams([tag for _, tag in pos_tags], 2)
    trigram_pos_seqs = ngrams([tag for _, tag in pos_tags], 3)
    
    #print(not_common_ngrams_list)

    # concatenate the three n-gram sequences into one
    all_ngram_pos = chain(unigram_pos_seqs, bigram_pos_seqs, trigram_pos_seqs)

    # create a list to store the counts of each n-gram in the not_common_ngrams_list
    pos_counts = []

    # Converting all_ngram_pos to a list and printing the first 10 n-grams
    all_ngrams_list = list(all_ngram_pos)
    #print(all_ngrams_list[:10])

    pos_counts = []
    
    for pos_ngram_list, top_pos_ngrams in zip([all_ngrams_list],
                                              [not_common_ngrams_list]):
        for pos_ngram in top_pos_ngrams:
            count = 0
            for top_pos in pos_ngram_list:
                if pos_ngram == top_pos:
                    count += 1
            pos_counts.append(count)

    
    # Tokenize the sentence into lowercase words
    tokens = nltk.word_tokenize(sentence.lower(), language='french')

    # Count the occurrences of each word in the sentence
    word_counts = Counter(tokens)

    # Create a feature vector based on the number of occurrences of the top 100 stopwords
    top_function_vector = []
    for stop_word in top_function_words:
        count = word_counts.get(stop_word, 0)
        top_function_vector.append(count)
           
    # count the occurrences of each punctuation mark
    punctuation_marks = ['.', ',', ';', ':', '?', '!', '«', '»', '-', '–', '—', '(', ')', '[', ']', '{', '}', '/', '\\', '...', "'", '"']
    
    punctuation_counts = []
    for mark in punctuation_marks:
        count = sentence.count(mark)
        punctuation_counts.append(count)

    #feature_vector = punctuation_counts + top_function_vector + pos_counts

    # create scaler objects for each feature type
    punc_scaler = StandardScaler()
    top_func_scaler = StandardScaler()
    pos_scaler = StandardScaler()

   # convert to NumPy arrays and reshape to have one column
    punctuation_counts_reshaped = np.array(punctuation_counts).reshape(-1, 1)
    top_function_vector_reshaped = np.array(top_function_vector).reshape(-1, 1)
    pos_counts_reshaped = np.array(pos_counts).reshape(-1, 1)
    
    # fit and transform each feature type separately
    punc_counts_scaled = punc_scaler.fit_transform(punctuation_counts_reshaped)
    top_func_scaled = top_func_scaler.fit_transform(top_function_vector_reshaped)
    pos_counts_scaled = pos_scaler.fit_transform(pos_counts_reshaped)
    
    # concatenate the scaled features
    feature_vector = np.concatenate((punc_counts_scaled, top_func_scaled, pos_counts_scaled), axis=0)
    
    # convert back to a 1D array and return
    return feature_vector.flatten()

# create feature vectors for each sentence
df['features'] = df['chunks'].apply(create_feature_vector)
test_df['features'] = test_df['chunks'].apply(create_feature_vector)

142
900


In [ ]:
len(test_df['features'][150]) #222 num=60 # 184 num = 25

214

In [ ]:

X_train = np.array(df['features'].tolist())
X_test = np.array(test_df['features'].tolist())

y_train = df['Author_name']
y_test = test_df['Author_name']

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

le = LabelEncoder()
y_test_encoded = le.fit_transform(y_test)

# create and train the SVM model
svm = SVC(kernel='linear')
svm.fit(X_train, y_train_encoded)

# Make predictions on the test set and calculate evaluation metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test_encoded, y_pred)
f1 = f1_score(y_test_encoded, y_pred, average='weighted')
precision = precision_score(y_test_encoded, y_pred, average='weighted')
recall = recall_score(y_test_encoded, y_pred, average='weighted')

# Print evaluation metrics
print("SVM accuracy:", accuracy)
print("SVM F1-score:", f1)
print("SVM Precision:", precision)
print("SVM Recall:", recall)

SVM accuracy: 0.8933685003767897
SVM F1-score: 0.8914358962714376
SVM Precision: 0.8956220914534114
SVM Recall: 0.8933685003767897


In [ ]:
print(classification_report(y_test_encoded, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90       567
           1       0.62      0.80      0.70       237
           2       0.95      0.85      0.90       736
           3       0.23      0.18      0.20        91
           4       0.95      1.00      0.98      1023

    accuracy                           0.89      2654
   macro avg       0.73      0.74      0.73      2654
weighted avg       0.89      0.89      0.88      2654



num = 50 has the best result

In [ ]:

num = 60

SVM accuracy: 0.8933685003767897
SVM F1-score: 0.8914358962714376
SVM Precision: 0.8956220914534114
SVM Recall: 0.8933685003767897

num = 40

SVM accuracy: 0.8858327053504145
SVM F1-score: 0.8847407636591803
SVM Precision: 0.887831478861173
SVM Recall: 0.8858327053504145


              precision    recall  f1-score   support

           0       0.91      0.89      0.90       567
           1       0.62      0.80      0.70       237
           2       0.95      0.85      0.90       736
           3       0.23      0.18      0.20        91
           4       0.95      1.00      0.98      1023

    accuracy                           0.89      2654
   macro avg       0.73      0.74      0.73      2654
weighted avg       0.89      0.89      0.88      2654

num = 50

SVM accuracy: 0.8948756593820648
SVM F1-score: 0.8922313961288101
SVM Precision: 0.8936182696795476
SVM Recall: 0.8948756593820648


             precision    recall  f1-score   support

           0       0.90      0.90      0.90       567
           1       0.63      0.78      0.70       237
           2       0.95      0.87      0.91       736
           3       0.35      0.21      0.26        91
           4       0.96      1.00      0.98      1023

    accuracy                           0.89      2654
   macro avg       0.76      0.75      0.75      2654
weighted avg       0.89      0.89      0.89      2654


num = 70

SVM accuracy: 0.892991710625471
SVM F1-score: 0.8904459396508708
SVM Precision: 0.8912335897834857
SVM Recall: 0.892991710625471

              precision    recall  f1-score   support

           0       0.90      0.91      0.90       567
           1       0.64      0.77      0.70       237
           2       0.95      0.86      0.90       736
           3       0.30      0.20      0.24        91
           4       0.96      1.00      0.98      1023

    accuracy                           0.89      2654
   macro avg       0.75      0.75      0.74      2654
weighted avg       0.89      0.89      0.89      2654




